Basic code for antibody-based simulations
===========

## basic imports ##

In [1]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from mdtraj.reporters import *
from functional import seq

from pathlib import Path
import sys
from sys import stdout
import inspect
from pycomfort.files import *

In [2]:
debug_local = True#False
local = Path("..").resolve()
code = local / "mm"
data  = local / "data"
input = data / "input"
output = data / "output"

if debug_local and code.exists():
    sys.path.insert(0, code.as_posix())
    print("extending pathes with local yspecies")
    print(sys.path)
    %load_ext autoreload
    %autoreload 2

extending pathes with local yspecies
['/data/sources/antibody-mm/mm', '/data/sources/antibody-mm/notebooks', '/home/antonkulaga/micromamba/envs/antibody-mm/lib/python39.zip', '/home/antonkulaga/micromamba/envs/antibody-mm/lib/python3.9', '/home/antonkulaga/micromamba/envs/antibody-mm/lib/python3.9/lib-dynload', '', '/home/antonkulaga/micromamba/envs/antibody-mm/lib/python3.9/site-packages', '/home/antonkulaga/micromamba/envs/antibody-mm/lib/python3.9/site-packages/IPython/extensions', '/home/antonkulaga/.ipython']


In [3]:
children(input).for_each(print)

/data/sources/antibody-mm/data/input/RA
/data/sources/antibody-mm/data/input/1ADQ


## Setting simulation parameters ##

In [5]:
def make_simulation(path: Path, delete_water: bool = True, add_hydrogens: bool = True):
    pdb = PDBFile(path.as_posix())
    modeller = Modeller(pdb.topology, pdb.positions)
    forcefield = ForceField('amber14/protein.ff14SB.xml', 'amber14/tip3pfb.xml')
    temperature = 310 * kelvin
    integrator = LangevinMiddleIntegrator(temperature, 1 / picosecond, 0.002*picoseconds)
    if delete_water:
        modeller.deleteWater()
    if add_hydrogens:
        modeller.addHydrogens(forcefield)
        #forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
    system = forcefield.createSystem(modeller.topology, 
        nonbondedMethod=CutoffNonPeriodic,
        nonbondedCutoff=1.2*nanometer,
        switchDistance=1.0*nanometer, 
        constraints=HBonds, 
        implicitSolvent=app.GBn2, implicitSolventSaltConc=0.15*moles/liter
        )
    simulation = Simulation(pdb.topology, system, integrator)
    simulation.context.setPositions(modeller.positions)
    simulation.minimizeEnergy()
    return simulation

In [6]:
def with_reporters(simulation: Simulation, pdb: Path, where: Path = output,
                 report_interval: int = 1000, checkpoint_interval: int = 10000, 
                 #report_pdb: bool = True #pdb reporter can be heavy
               ): 
    #simulation.reporters.append(PDBReporter((where / f'{pdb.stem}.dcd').as_posix(), report_interval))
    simulation.reporters.append(DCDReporter((where / f'{pdb.stem}.dcd').as_posix(), report_interval))
    simulation.reporters.append(StateDataReporter((where / f'{pdb.stem}_log.tsv').as_posix(), report_interval, potentialEnergy=True, kineticEnergy=True, totalEnergy=True, step=True,temperature=True, separator='\t'))
    simulation.reporters.append(HDF5Reporter((where / f'{pdb.stem}_log.h5').as_posix(), report_interval, coordinates=True, time=True, cell=True, potentialEnergy=True, kineticEnergy=True, temperature=True))
    simulation.reporters.append(CheckpointReporter((where / f'{pdb.stem}_results.chk').as_posix(), checkpoint_interval))
    return simulation

In [17]:
def save_results(sim: Simulation, where: Path = output) -> Simulation:
    finalpositions = sim.context.getState(getPositions=True).getPositions()
    PDBFile.writeFile(sim.topology, finalpositions, open((where / f'{pdb_stem}.pdb').as_posix(), 'w'))
    sim.saveState('finalstate.xml')
    sim.saveCheckpoint('finalstate.chk')
    sim.saveState((output / f'{pdb_stem}_results.xml').as_posix())
    sim.saveCheckpoint((output / f'{pdb_stem}_results.chk').as_posix())
    return sim

### Loading folders and files ###

In [7]:
docking_results = Path("/data/samples/docking/results/")
best_docking_results = docking_results / "best"
tprint(best_docking_results)


best
	S5205Nr1-P2_IgG1Fc_H_top_30_heavy_chains
		3_17806_H_5_energy_-238.0523.pdb
		3_17806_H_35_energy_-224.0373.pdb
		FIXED_1_35744_H_64_energy_-263.5477.pdb
		FIXED_1_35744_H_1_energy_-254.21349.pdb
		1_35744_H_64_energy_-263.5477.pdb
		1_35744_H_1_energy_-254.21349.pdb
		2_27457_H_257_energy_-248.7721.pdb
		FIXED_1_35744_H_112_energy_-265.2682.pdb
		1_35744_H_112_energy_-265.2682.pdb
		2_27457_H_253_energy_-246.08788.pdb
		2_27457_H_244_energy_-284.44533.pdb
		3_17806_H_24_energy_-221.3707.pdb
	S5205Nr1-P2_PBMC_L_top_30_heavy_chains
		1_27244_H_30_energy_-263.64684.pdb
		1_27244_H_23_energy_-251.4212.pdb
		1_27244_H_297_energy_-260.30115.pdb


In [8]:
p2_auto = best_docking_results / "S5205Nr1-P2_IgG1Fc_H_top_30_heavy_chains"
p2_pbmc = best_docking_results / "S5205Nr1-P2_PBMC_L_top_30_heavy_chains"
tprint(p2_auto)

S5205Nr1-P2_IgG1Fc_H_top_30_heavy_chains
	3_17806_H_5_energy_-238.0523.pdb
	3_17806_H_35_energy_-224.0373.pdb
	FIXED_1_35744_H_64_energy_-263.5477.pdb
	FIXED_1_35744_H_1_energy_-254.21349.pdb
	1_35744_H_64_energy_-263.5477.pdb
	1_35744_H_1_energy_-254.21349.pdb
	2_27457_H_257_energy_-248.7721.pdb
	FIXED_1_35744_H_112_energy_-265.2682.pdb
	1_35744_H_112_energy_-265.2682.pdb
	2_27457_H_253_energy_-246.08788.pdb
	2_27457_H_244_energy_-284.44533.pdb
	3_17806_H_24_energy_-221.3707.pdb


In [9]:
pdb = p2_auto / "FIXED_1_35744_H_112_energy_-265.2682.pdb"


### Making simulation

In [10]:
sim = make_simulation(pdb)

In [11]:
sim_ext = with_reporters(sim, pdb, output)
sim_ext

## Running the simulation ##

In [12]:
steps = 50000 #50000000 # to get 10 nanoseconds

In [14]:
sim.step(steps)

/home/antonkulaga/micromamba/envs/antibody-mm/lib/python3.9/site-packages/mdtraj/utils/validation.py:115: TypeCastPerformanceWarning: Casting xyz dtype=float64 to <class 'numpy.float32'> 
  warnings.warn("Casting %s dtype=%s to %s " % (name, val.dtype, dtype),


AttributeError: 'PosixPath' object has no attribute 'write'

In [14]:
finalpositions = sim_ext.context.getState(getPositions=True).getPositions()
PDBFile.writeFile(sim_ext.topology, finalpositions, open((output / f'{pdb_stem}.pdb').as_posix(), 'w'))

sim_ext.saveState('finalstate.xml')
sim_ext.saveCheckpoint('finalstate.chk')

sim_ext.saveState((output / f'{pdb_stem}_results.xml').as_posix())
sim_ext.saveCheckpoint((output / f'{pdb_stem}_results.chk').as_posix())